In [1]:
from tkinter import *
from tkinter import filedialog
from tkinter import ttk
import re
import pandas as pd
import numpy as np
import regex
import os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract
from pytesseract import Output, TesseractError
import glob
import PyPDF2 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import openpyxl
from openpyxl import load_workbook
import xlwings as xw

import time
import shutil
import tempfile

from datetime import date

In [10]:
####### Dask configuration #######
from multiprocessing.pool import ThreadPool
import dask
import dask.bag as db
import multiprocessing as mp
cpu_count = mp.cpu_count()-1
dask.config.set(pool=ThreadPool(cpu_count))

# Datum von heute
heute = date.today()
heute = heute.strftime("%d%m%Y")

# User Interface
window = Tk()
window.title('Hallo DM')
window.geometry('800x400')
window.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
window.config(bg='#eeeee4')

# Text Erklärung
wozu = Label(window, text=
             'Hallo DM!\n Mit diesem Programm werden die Infos aus den Labor-Befunden extrahiert.',
            bg = '#eeeee4', font=('Ink free',15))
wozu.grid(row=1, column=1, columnspan=2)

# Allgemeine Funktionen
def new_list(input_list, zeichen):
    """ Erstellen einer neuen leeren Liste mit derselben Länge wie Input
        input_list (Liste mit der gewünschten Länge)
        zeichen (default Listeneintrag)"""
    new = [zeichen]*len(input_list)
    return(new)

def open_dm_file(name, path):
    """ Öffnet eine der Dateien des DMs.
        Dafür werden der Name der Datei und der Pfad als Argumente übergeben."""
    files = glob.glob(path + '\*.xlsx*')
    for i in range(len(files)):
        if name in files[i]:
            path = files[i]
            break
    try:
        excel = pd.read_excel(path)
    # falls Datei geöffnet ist:
    except:
        # Generate a temporary directory to store the copied file
        temp_dir = tempfile.mkdtemp()        
        # Generate a temporary file name with a .xlsx extension in the temporary directory
        temp_file_path = tempfile.mktemp(suffix='.xlsx', dir=temp_dir)        
        # Copy the open Excel file to the temporary location
        shutil.copy2(path, temp_file_path)
        excel = pd.read_excel(temp_file_path)    
        shutil.rmtree(temp_dir)
    return(excel)

def open_dm_file_sheet(name, path, sheetname):
    """ Öffnet besonderes sheet der Datei des DMs.
        Dafür werden der Name der Datei, Pfad und sheetname als Argumente übergeben."""
    files = glob.glob(path + '\*.xlsx*')
    for i in range(len(files)):
        if name in files[i]:
            path = files[i]
            break
    try:
        excel = pd.read_excel(path, sheet_name = sheetname)
    # falls Datei geöffnet ist:
    except:
        # Generate a temporary directory to store the copied file
        temp_dir = tempfile.mkdtemp()        
        # Generate a temporary file name with a .xlsx extension in the temporary directory
        temp_file_path = tempfile.mktemp(suffix='.xlsx', dir=temp_dir)        
        # Copy the open Excel file to the temporary location
        shutil.copy2(path, temp_file_path)
        excel = pd.read_excel(temp_file_path, sheet_name = sheetname)    
        shutil.rmtree(temp_dir)
    return(excel)

def decimal_value(value):
    """ Wandelt Zahl (z. Bsp. 0.17) in Dezimalzahl mit 2 oder 3 Nachkommastellen um. """
    value = float(value)
    value = "{:.3f}".format(value).replace('.', ',')
    if value[-1] == '0':
        value = value[:-1]
    return(value)

def price(p):
    """ Bearbeitet Preis in der Preis-Spalte """
    try:
        p = float(p)
        p = "{:.2f}".format(p).replace('.', ',')
    except:
        p = ''
    return(p)

def add_to_befunddokumentation(df):
    files = glob.glob('*.xlsx*')
    for i in range(len(files)):
        if 'Befunddokumentation zum Import in IDA' in files[i]:
            path = files[i]
            if '~$' in path:
                path = path.replace('~$', '')
            
    # nur eine Spalte einlesen, da es sonst länger dauert
    wb = xw.Book(path)
    ws = wb.sheets['Befundliste']
    start_row = ws.range('H11').end('down').row +1

    # Assuming you have defined and populated the DataFrame df

    cols = [5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 26]

    for r_idx, row in enumerate(df.values):
        for c_idx, cell_value in enumerate(row):
            c_idx = cols[c_idx] + 1
            ws.cells(start_row + r_idx, c_idx).number_format = "@"
            ws.cells(start_row + r_idx, c_idx).value = cell_value
            
    wb.save()

#################################### Labor 28 ######################################################    
def go_homepage():
    """ Funktion führt zurück zur Startseite"""
    root2.grid_forget()
    back_to_homepage.grid_forget()
    labor_28.grid(row=5, column=1, pady=1, columnspan=2)
    ladr.grid(row=6, column=1, pady=1, columnspan=2)
    synlab.grid(row=7, column=1, pady=1, columnspan=2)
    spranger.grid(row=8, column=1, pady=1, columnspan=2)

def labor28():
    
    global done, file_path
    
    try:
        done.grid_forget()
    except:
        pass
    
    
    # Ladebalken
    space = Label(text='')
    space.grid(row=13, column=1,)
    progress_bar = ttk.Progressbar(window, orient="horizontal", length=250, mode="determinate")
    progress_bar.grid(row=14, column=1, columnspan=5)    
            
    # pdf auswählen
    file_path = filedialog.askopenfilenames(title = "Datei auswählen (Labor 28)")
    files = list(file_path)
    
    if files == []:
        progress_bar.grid_forget()
        return()
    
    progress_bar.start()
    
    text_list = []
    for file in files:            
        pdfFileObj = open(file,'rb')
        pdfReader = PyPDF2.PdfReader(pdfFileObj)

        totalpages = len(pdfReader.pages)

        for i in range(totalpages):
            pageObj = pdfReader.pages[i]
            text = pageObj.extract_text()
            text_list.append(text)

    for i in range(len(text_list)):
        text_list[i] = text_list[i].split('\n')
        
    progress_bar.step(20)
    window.update()
    
    # aus allen pdf-Befunden eine große pdf erstellen
    # Create a PDF merger object
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in files:
        with open(pdf_file, 'rb') as pdf:
            pdf_merger.append(pdf)
    output_pdf = 'labor28_{}.pdf'.format(heute)
    # Write the merged PDF to the output file
    with open(output_pdf, 'wb') as output:
        pdf_merger.write(output)
    # Close the merger
    pdf_merger.close()
    
    progress_bar.step(10)
    window.update()
    
    # benötigte DM-Dateien öffnen
    feldcode = open_dm_file('Methoden Feldcodes', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen')

    # aus den DM-Dateien Infos extrahieren, die später benötigt werden
    def method_from_feldcode(x):
        return(' ' + x.split('=')[1].split(' ')[0])
    
    test_methods = [method_from_feldcode(x) for x in list(feldcode['Methoden']) if len(method_from_feldcode(x)) < 10]

    # weitere Methoden hinzufügen
    more_methods = [' HEXO', ' FARBT', ' PHOT', ' KOAG', 'IB']
    for m in more_methods:
        test_methods.append(m)
    for i in range(len(test_methods)):
        if ' Biuret' == test_methods[i]:
            test_methods[i] = test_methods[i].replace(' Biuret', ' BIURET')
    test_methods = sorted(test_methods, key=len, reverse = True)
    
    sample_id = new_list(text_list, '')
    datum = new_list(text_list, '')
    parameter_total = new_list(text_list, '')
    method = new_list(text_list, '')
    remarks_cap = new_list(text_list, '')

    for i in range(len(text_list)):
        for j in range(len(text_list[i])):
            if 'Patient:' in text_list[i][j]:
                sample_id[i] = 'I' + text_list[i][j].split('Patient:')[1].split(',')[0].strip().split('+')[0].split('_')[0][:-3]
                if sample_id[i][1] == 'I':
                    sample_id[i] = sample_id[i][1:]
                i_start = j + 1
                text_list[i] = text_list[i][i_start:]
                datum[i] = text_list[i][0].split(' ')[-1]
                break
    # unnötige Infos über der Spalte "Abnahmedatum" aus der Liste entfernen
    for i in range(len(text_list)):            
        for j in range(len(text_list[i])):
            if 'Abnahmedatum' in text_list[i][j]:
                i_start = j + 1
                text_list[i] = text_list[i][i_start:]
                break  
    # unnötige Infos unter der Spalte "Befund freigegeben" aus der Liste entfernen
    for i in range(len(text_list)): 
        for j in range(len(text_list[i])):
            if 'Befund freigegeben' in text_list[i][j]:
                i_end = j
                text_list[i] = text_list[i][:i_end]
                break    
    # Parameter-Zeile anhand der Testmethode finden, diese steht immer am Ende der Zeile
    # gleichzeitig nach Grenzbereich suchen
    grenze = ['Grenzbereich', 'Graubereich']
    for i in range(len(text_list)):
        parameter = []
        test_m = []
        cap_klasse = []
        for j in range(len(text_list[i])):
            # überschüssige Informationen in den Zeilen entfernen
            if 'negativ' not in text_list[i][j]:
                if 'grenzwertig ' in text_list[i][j]:
                    text_list[i][j] = text_list[i][j].replace('grenzwertig ', '')
                if ' grenzwertig' in text_list[i][j]:
                    text_list[i][j] = text_list[i][j].replace(' grenzwertig', '')
                if 'grenzwertig' in text_list[i][j]:
                    text_list[i][j] = text_list[i][j].replace('grenzwertig', '')
            for test in test_methods:
                if (test == text_list[i][j][-len(test):]) | (test.upper() == text_list[i][j][-len(test):]):
                    text_list[i][j] = text_list[i][j][:text_list[i][j].index(test)]
                    test_m.append(test[1:])
                    # Falls es sich um Allergie-Parameter handelt, steht in der nächsten Zeile
                    # die CAP-Klasse mit Referenzbereich sowie die CAP-Klasse
                    allergie_ref = ''
                    cap = ''
                    try:
                        if 'CAP-Klasse' in text_list[i][j+1]:
                            allergie_ref = '  ' + text_list[i][j+1].split('(')[1].split(')')[0]
                            cap = 'CAP/RAST/EAST-class: ' +  text_list[i][j+1].split(' ')[1].split(' ')[0]
                    except:
                        pass
                    cap_klasse.append(cap)
                    
                    # nach Grenzbereich in den nächsten Zeilen suchen,
                    # falls dieser nicht in derselben Zeile steht
                    para = text_list[i][j] + allergie_ref
                    para_grenze = ''
                    for l in range(j, len(text_list[i])):
                        for g in grenze:
                            if (g in text_list[i][l]) & (' ' + text_list[i][l].split(' ')[-1].strip() not in test_methods):
                                if (' ' + text_list[i][l-1].split(' ')[-1].strip() not in test_methods) & (' ' + text_list[i][j+1].split(' ')[-1].strip() not in test_methods):
                                    para_grenze = 'marker' + text_list[i][l]
                                    break
                    # nach positiv-Bereich in den nächsten Zeilen suchen
                    para_p = ''
                    for l in range(j, len(text_list[i])):
                        if ('positiv' in text_list[i][l]) & ('>' in text_list[i][l]) & (' ' + text_list[i][l].split(' ')[-1] not in test_methods):
                            # Check, dass Bemerkung nicht zu einem späteren Parameter gehört
                            positiv_index = l
                            flag_positiv = True
                            for positiv_i in range(j+1, positiv_index):
                                if ' ' + text_list[i][positiv_i].split(' ')[-1] in test_methods:
                                    flag_positiv = False
                            if flag_positiv:
                                para_p = 'marker' + text_list[i][positiv_index]
                        # falls es sich nicht um einen Allergie-Parameter handelt
                        if allergie_ref == '':
                            para = text_list[i][j] + para_p + para_grenze
                    
                    # manchmal steht in der Spalte Richtwert der Grenzbereich
                    # und der eigentliche Richtwert steht in der nächsten Zeile
                    for g in grenze:
                        if g in text_list[i][j]:
                            # marker vor dem Grenzbereich setzen
                            text = text_list[i][j].split('-')
                            # wenn 'Grenzbereich' im vorletzten Eintrag steht,
                            # dann marker vor 'Grenzbereich' setzen
                            if g in text[-2]:
                                text = text_list[i][j].split(g)[0] + 'marker' + g + text_list[i][j].split(g)[1]                            
                                para = text                                
                            # wenn 'Grenzbereich' im letzten Eintrag steht,
                            # dann marker entsprechend setzen
                            elif g in text[-1]:
                                t = text[-2].split(' ')
                                t.insert(-1, 'marker')
                                t = ' '.join(t)
                                text[-2] = t
                                text = '-'.join(text)
                                para = text                                                        
                            # nach reference in der nächsten Zeile suchen und vor 'marker' einfügen
                            if ('(' in text_list[i][j+1]) & (')' in text_list[i][j+1]):
                                ref = text_list[i][j+1].split('(')[-1].split(')')[0]
                                # Reihenfolge reference und unit kontrollieren
                                ref = ref.split(' ')
                                for p in ['/', 'mol', 'CE', '%']:                                    
                                    if p in ref[-1]:
                                        ref = ' '.join(ref[::-1])
                                para = para.split('marker')
                                para[-2] = para[-2] + ref
                                para = 'marker'.join(para)
                            break
                    
                    # nach 'positiv' im Richwert suchen
                    if ('positiv' in text_list[i][j]) & ('>' in text_list[i][j]):
                        if ('(' in text_list[i][j+1]) & (')' in text_list[i][j+1]):
                            # 'marker' einfügen
                            text = text_list[i][j].split('positiv')
                            # wenn '>' im letzen Eintrag steht, dann steht marker vor 'positiv'
                            if '>' in text[-1]:
                                text[-2] = text[-2] + 'marker' 
                                text = 'positiv'.join(text)
                                para = text
                            # wenn '>' im vorletzten Eintrag steht, marker entsprechend setzen
                            elif '>' in text[-2]:
                                text = text.split('positiv')
                                text[-2] = text[-2].split('>')
                                text[-2][-2] = test[-2][-2] + 'marker'
                                text[-2] = '>'.join(text[-2])
                                text = 'positiv'.join(text)
                                para = text
                            # reference vor marker einfügen
                            ref = text_list[i][j+1].split('(')[-1].split(')')[0]
                            # Reihenfolge reference und unit kontrollieren
                            ref = ref.split(' ')
                            for p in ['/', 'mol', 'CE', '%']:                                    
                                if p in ref[-1]:
                                    ref = ' '.join(ref[::-1])
                            para = para.split('marker')
                            para[-2] = para[-2] + ref
                            para = 'marker'.join(para)                                                                    
                    parameter.append(para)
                    break
                            
            # manchmal steht hinter einer Titer-Vermessung keine Methode
            # wie z. Bsp. bei p-ANCA-Titer
            if 'Titer' in text_list[i][j]:
                parameter.append(text_list[i][j])
                test_m.append('')

            method[i] = test_m
            parameter_total[i] = parameter
            remarks_cap[i] = cap_klasse

    progress_bar.step(10)
    window.update()
    
    # falls hinter Titer doch eine Methode steht, wird diese Zeile doppelt in parameter_total übernommen
    # eine Zeile löschen
    drop_index = []
    for i in range(len(parameter_total)):        
        for j in range(len(parameter_total[i])-1):
            if ('Titer' in parameter_total[i][j]) & ('Titer' in parameter_total[i][j+1]):
                if method[i][j+1] == '':
                    if parameter_total[i][j] == parameter_total[i][j+1]:
                        drop_index.append(j+1)

        if drop_index != []:
            parameter_total[i] = [el for l, el in enumerate(parameter_total[i]) if l not in drop_index]
            method[i] = [el for l, el in enumerate(method[i]) if l not in drop_index]
            
    # sample_id, parameter_total, datum und method auf dieselbe Dimension bringen
    multi_list = []
    for i in range(len(parameter_total)):
        multi_list.append(len(parameter_total[i]))

    sample_final = []
    datum_final = []
    for x,y in zip(sample_id, multi_list):
        sample_final.extend([x]*y)
    for x,y in zip(datum, multi_list):    
        datum_final.extend([x]*y)

    parameter_final = [item for sublist in parameter_total for item in sublist]
    method_final = [item for sublist in method for item in sublist]
    remarks_cap_final = [item for sublist in remarks_cap for item in sublist]
    
    # Liste mit remarks to reference erstellen und mit Boderline-Werten ergänzen
    remarks_to_reference = new_list(parameter_final, '')
    for i in range(len(remarks_to_reference)):
        for g in grenze:
            if (g in parameter_final[i]) & ('marker' in parameter_final[i]):
                rem = parameter_final[i].split('marker')            
                # Grenzbereich
                for g in grenze:
                    for j in range(len(rem)):
                        if g in rem[j]:
                            rem = rem[j]                        
                            rem = rem.replace(',', '.')
                            rem = rem.split(' ')
                            rem1 = ''
                            rem2 = ''
                            for l in range(len(rem)):
                                try:
                                    rem1 = rem[l]
                                    rem1 = decimal_value(rem1)
                                    rem1_i = l
                                    break
                                except:
                                    pass
                            for k in range(rem1_i+1, len(rem)):
                                try:
                                    rem2 = rem[k]
                                    rem2 = decimal_value(rem2)
                                    if rem2 != '':
                                        break
                                except:
                                    pass
                            if (rem1 != '') & (rem2 != ''):
                                rem = 'borderline: ' + str(rem1) + ' - ' + str(rem2)
                            remarks_to_reference[i] = rem
                parameter_final[i] = 'marker'.join(parameter_final[i].split('marker')[:-1])
                break
    # positiv
    for i in range(len(parameter_final)):
        if 'marker' in parameter_final[i]:
            rem = parameter_final[i].split('marker')
            for j in range(len(rem)):
                if ('positiv' in rem[j]) & ('>' in rem[j]):
                    rem = rem[j].split(' ')
                    rem_f = ''
                    for r in rem:
                        try:
                            rem_f = decimal_value(r)
                        except:
                            pass
            if '=' in rem[j]:
                rem_f = 'positive: >= ' + rem_f
            else:
                rem_f = 'positive: > ' + rem_f
            
            if remarks_to_reference[i] != '':
                remarks_to_reference[i] = remarks_to_reference[i] + ', ' + rem_f
            else:
                remarks_to_reference[i] = rem_f
            parameter_final[i] = parameter_final[i].split('marker')[0]
    
    # reference range rauslesen
    reference_range = new_list(parameter_final, '')
    for i in range(len(parameter_final)):
        # nach '<' und '<=' gucken
        for kleiner in ['<', '<=', '< =']:
            if kleiner in parameter_final[i]:
                reference = parameter_final[i].split(kleiner)
                # wenn der letzte Eintrag in ein float umgewandelt werden kann,
                # handelt es sich um eine reference range
                if ':' not in reference[-1]:
                    try:
                        ref = reference[-1].strip()
                        ref = decimal_value(ref)
                        # richtig formatieren
                        if kleiner == '< =':
                            kleiner = '<='
                        reference_range[i] = kleiner + ' ' + ref
                        # wenn '<' doppelt in der Zeile vorkommt, wird falsch gesplitted
                        if len(reference) <= 2:
                            parameter_final[i] = reference[0].strip()
                        else:
                            parameter_final[i] = '{}'.join(reference[:-1]).strip().format(kleiner)
                        # manche Allergie-Parameter haben komische reference range
                        # daher muss die Parameter-Liste darauf angepasst werden
                        if parameter_final[i][-2:] == ' -':
                            parameter_final[i] = ' '.join(parameter_final[i].split(' -')[0].split(' ')[:-1]).strip()
                    except:
                        pass
                    # richtig formatieren
                    if kleiner == '< =':
                        parameter_final[i] = parameter_final[i].replace('< =', '<=')
                    
                # manchmal steht auch ein Wert wie 1:10, wobei es sich auch um eine reference range handelt
                else:
                    try:
                        ref = reference[-1].split(':')
                        ref1 = int(ref[-2].strip())
                        ref2 = int(ref[1].strip())
                        ref = str(ref1) + ': ' + str(ref2)
                        reference_range[i] = kleiner + ' ' + ref
                        parameter_final[i] = ' '.join(parameter_final[i].split(kleiner)[:-1]).strip()
                    except:
                        pass
                # manchmal steht noch 'Perzentil' in der reference range
                if 'Perzentil)' in reference[-1]:
                    try:
                        ref = reference[-1].split('(')[0].strip()
                        ref = decimal_value(ref)
                        reference_range[i] = kleiner + ' ' + ref + ' (' + reference[-1].split('(')[-1]
                        parameter_final[i] = ' '.join(parameter_final[i].split(kleiner)[:-1]).strip()
                    except:
                        pass           
        # nach '>' und '>=' gucken
        for groesser in ['>', '>=', '> =']:
            if groesser in parameter_final[i]:
                reference = parameter_final[i].split(groesser)
                # wenn der letzte Eintrag in ein float umgewandelt werden kann,
                # handelt es sich um eine reference range
                try:
                    ref = reference[-1].strip()
                    ref = decimal_value(ref)
                    # richtig formatieren
                    if groesser == '> =':
                        groesser = '>='
                    reference_range[i] = groesser + ' ' + ref
                    # wenn '>' doppelt in der Zeile vorkommt, wird falsch gesplitted
                    if len(reference) <= 2:
                        parameter_final[i] = reference[0].strip()
                    else:
                        parameter_final[i] = '{}'.join(reference[:-1]).strip().format(groesser)
                    # manche Allergie-Parameter haben komische reference range
                    # daher muss die Parameter-Liste darauf angepasst werden
                    if parameter_final[i][-2:] == ' -':
                        parameter_final[i] = ' '.join(parameter_final[i].split(' -')[0].split(' ')[:-1]).strip()
                except:
                    pass

        # nach '-' gucken nur wenn Eintrag in reference_range noch leer ist
        if reference_range[i] == '':
            if '-' in parameter_final[i]:
                reference = parameter_final[i].split('-')
                # wenn der Wert vor und nach dem '-' in ein float umgewandelt werden kann,
                # handelt es sich um eine reference range
                try:
                    ref2 = reference[-1].strip()
                    ref2 = decimal_value(ref2)
                    ref1 = reference[-2].strip().split(' ')[-1]
                    ref1_1 = decimal_value(ref1)
                    ref = ref1_1 + ' - ' + ref2
                    reference_range[i] = ref
                    # manchmal ist der Richtwert derselbe wie der Messwert, dann wird falsch gesplitted
                    if len(parameter_final[i].split(ref1)) <= 2:
                        parameter_final[i] = parameter_final[i].split(ref1)[0].strip()
                    else:
                        parameter_final[i] = '{}'.join(parameter_final[i].split(ref1)[:-1]).strip().format(ref1)
                except:
                    pass
        # nach 'negativ' gucken, wenn reference_range[i] leer ist
        parameter_final[i] = parameter_final[i].strip()
        if reference_range[i] == '':
            if parameter_final[i][-len('negativ'):] == 'negativ':
                if ('positiv' in parameter_final[i][:-len('negativ')]) | ('negativ' in parameter_final[i][:-len('negativ')]):
                    parameter_final[i] = parameter_final[i][:-len('negativ')].strip()
                    reference_range[i] = 'negative'
                else:
                    reference_range[i] = 'negative'
            # manchmal steht kein 'negativ', wenn als measured value 'positiv' steht
            for pos in ['pos.', 'positiv']:
                if parameter_final[i][-len(pos):] == pos:
                    reference_range[i] = 'negative'
                    
    progress_bar.step(10)
    window.update()
                    
    # Liste mit remarks values erstellen
    drop_index = []
    remarks = new_list(parameter_final, '')
    for i in range(len(parameter_final)):
        if 'HEp-2-Zellmuster' in parameter_final[i]:
            rem = parameter_final[i].split('HEp-2-Zellmuster ')[-1]
            drop_index.append(i)
            remarks[i-1] = rem
        elif 'TSH (sensitiv)' in parameter_final[i]:
            remarks[i] = 'sensitive'
        if remarks_cap_final[i] != '':
            remarks[i] = remarks_cap_final[i]
        
    if drop_index != []:
        sample_final = [el for l, el in enumerate(sample_final) if l not in drop_index]
        datum_final = [el for l, el in enumerate(datum_final) if l not in drop_index]
        parameter_final = [el for l, el in enumerate(parameter_final) if l not in drop_index]
        reference_range = [el for l, el in enumerate(reference_range) if l not in drop_index]
        remarks = [el for l, el in enumerate(remarks) if l not in drop_index]
        method_final = [el for l, el in enumerate(method_final) if l not in drop_index]
        remarks_to_reference = [el for l, el in enumerate(remarks_to_reference) if l not in drop_index]
        
    # Wenn ein Eintrag in reference_range immer noch leer ist, dann kann es sein,
    # dass der Referenzwert erst in den folgenden Zeilen angegeben wird (wie z. Bsp. bei Thyreoglobulin)
    for i in range(len(reference_range)):
        if reference_range[i] == '':
            sample_search = sample_final[i]
            text_list_search = text_list[sample_id.index(sample_search)]
            for j in range(len(text_list_search)):
                if 'Wiederfindungsrate' in text_list_search[j]:
                    try:
                        ref = text_list_search[j-1]
                        ref = ref.split('-')
                        ref2 = ref[-1].strip()
                        ref2 = decimal_value(ref2)
                        ref1 = ref[-2].strip().split(' ')[-1]
                        ref1 = decimal_value(ref1)
                        ref = ref1 + ' - ' + ref2
                        reference_range[i] = ref
                    except:
                        pass
    
    progress_bar.step(10)
    window.update()
                    
    # Liste mit Einheiten erstellen
    units = new_list(parameter_final, '-')
    for i in range(len(units)):
        for p in ['/', 'mol', 'CE', '%', 'CU']:
            if p in parameter_final[i].split(' ')[-1]:
                if p == '/':
                    if ('negativ' in parameter_final[i].split('/')[0][-len('negativ'):]) | ('positiv' in parameter_final[i].split('/')[0][-len('positiv'):]):
                        continue
                units[i] = parameter_final[i].split(' ')[-1]
                # in manchen Fällen steht zwischen Einheit und Messwert kein Leerzeichen
                match = re.search(r'(\d+)([^\d]*)', units[i])
                number = ''
                if match:
                    number = ' ' + match.group(1)
                    units[i] = match.group(2)
                parameter_final[i] = ' '.join(parameter_final[i].split(' ')[:-1])+number
        if units[i] == 'CE':
            units[i] = 'CU'
        if 'IE' in units[i]:
            units[i] = units[i].replace('IE', 'IU')
        if units[i] == '-':
            if 'Titer' in parameter_final[i]:
                units[i] = 'Titer'
            if ':' in reference_range[i]:
                units[i] = 'Titer'
                
    # Liste mit den Messwerten erstellen
    measured_value = new_list(parameter_final, '')
    zeichen = ['<', '<=', '>', '>=']
    for i in range(len(measured_value)):
        if 'stark pos.' in parameter_final[i]:
            measured_value[i] = 'high positive'
            parameter_final[i] = parameter_final[i].split('stark pos.')[0].strip()
        elif 'schwach pos.' in parameter_final[i]:
            measured_value[i] = 'low positive'
            parameter_final[i] = parameter_final[i].split('schwach pos.')[0].strip() 
        else:
            if 'grenzwertig' in parameter_final[i]:
                if 'negativ' in parameter_final[i]:
                    measured_value[i] = 'borderline'
                    parameter_final[i] = parameter_final[i].split('grenzwertig')[0].strip()                    
            else:
                value = parameter_final[i].split(' ')[-1]
                if value == 'positiv':
                    value = 'positive'
                elif value == 'negativ':
                    value = 'negative'
                elif ':' in value:
                    value = value.split(':')[0] + ': ' + value.split(':')[1]
                for z in zeichen:
                    if z in value:
                        if ':' in value:
                            value = value.replace(z, z + ' ')
                        else:
                            value = value.split(z)[-1]
                            value = decimal_value(value)
                            value = z + ' ' + value

                try:
                    value = decimal_value(value)
                except:
                    pass            

                measured_value[i] = value
                parameter_final[i] = ' '.join(parameter_final[i].split(' ')[:-1])

                # Die Einheit 'Index' kann versehentlich auch als Messwert abgelesen werden
                # korrigieren
                if (measured_value[i] == 'Index') & (units[i] == '-'):
                    units[i] = 'Index'
                    value = parameter_final[i].split(' ')[-1].strip()
                    try:
                        value = decimal_value(value)
                    except:
                        pass
                    measured_value[i] = value
                    parameter_final[i] = ' '.join(parameter_final[i].split(' ')[:-1])
    
    progress_bar.step(10)
    window.update()
                
    # falls es sich um eine Fremdleisung handelt, werden die Parameter mit '* ' markiert.
    # Markierung aus dem Parameternamen entfernen
    for i in range(len(parameter_final)):
        if '*' in parameter_final[i]:
            parameter_final[i] = parameter_final[i].replace('*', '').strip()
                            
    # Die Parameter werden (sofern es geht) ins Englische übersetzt
    para_en_dict = {
        'nRNP/Sm-AK': 'anti-nRNP/Sm',
        'HDL-Cholesterin': 'High density lipoprotein-cholesterol (HDL-Cholesterol)',
        'Thyreoglobulin (hTG sensitiv)': 'Thyroglobulin (Tg) recovery rate',
        'LDL-Cholesterin': 'Low-Density-Lipoprotein-Cholesterin (LDL)',
        'Proteinase 3-AK (PR3-AK)': 'anti-Proteinase 3 (PR3)',
        'IgM-RF': 'Rheumatoid Factor (RF) IgM',
        'Toxoplasma-IgG-AK': 'anti-Toxoplasma IgG',
        'Toxoplasma-IgM-AK': 'anti-Toxoplasma IgM',
        'AMA (Antimitochondriale AK)': 'Antimitochondrial Antibodies (AMA)',
        'AMA-M2': 'Antimitochondrial Antibodies (AMA-M2)',
        'Non-HDL-Cholesterin': 'non-High-density-lipoprotein-cholesterol (non-HDL-C)',
        'GFR-Cystatin C': 'Glomerular filtration rate (GFR)',
        'Borrelien-AK (IgM)': 'anti-Borrelia IgM',
        'Borrelien-AK (IgG)': 'anti-Borrelia IgG',
        'Transglutaminase IgA-AK': 'anti-Tissue transglutaminase (tTG) IgA',
        'Gliadin IgG-AK': 'anti-Gliadin deamidated IgG',
        'Gliadin IgA-AK': 'anti-Gliadin deamidated IgA',
        'Endomysium IgA': 'anti-Endomysium Antibodies (EMA) IgA',
        'Endomysium IgG': 'anti-Endomysium Antibodies (EMA) IgG',
        'Endomysium IgA-AK': 'anti-Endomysium Antibodies (EMA) IgA',
        'Endomysium IgG-AK': 'anti-Endomysium Antibodies (EMA) IgG',
        'ANA-Titer': 'Antinuclear Antibodies (ANA-Titer)',
        'ANA': 'Antinuclear Antibodies (ANA)',
        'TSH (sensitiv)': 'Thyrotropin (TSH)',
        'TRAK': 'TSH Receptor Antibodies (TRAb)',
        'Elastase i. Stuhl': 'Elastase',
        'IgE': 'Total IgE',
        'AMA (Antimitochondriale AK)': 'Antimitochondrial Antibodies (AMA)',
        'Albumin': 'Albumin',
        'Helicobacter pylori Antigen': 'Helicobacter pylori Antigen',
        'Helicobacter pylori': 'Helicobacter pylori',
        'Helicobacter IgA AK': 'anti-Helicobacter IgA',
        'Helicobacter IgG AK': 'anti-Helicobacter IgG',
        'Zink': 'Zinc',
        'Ges. Eiweiß': 'Total Protein',
        'IgG': 'Total IgG',
        'IgG Gesamt': 'Total IgG',
        'IgM': 'Total IgM',
        'IgM Gesamt': 'Total IgM',
        'IgA': 'Total IgA',
        'IgA Gesamt': 'Total IgA',
        'Cyclisches Citrullin-Peptid (CCP) AAK': 'anti-cyclic citrullinated Peptide (CCP) IgG / anti-citrullinated Protein Antibodies (ACPA) IgG',
        'Scl-70-Antikörper': 'anti-Scl-70',
        'Cholesterin ges': 'Total Cholesterol',
        'Kreatinin': 'Creatinine',
        'SS-B (La)-AK': 'anti-SS-B (La)',
        'Jo-1-Autoantikörper': 'anti-Jo-1 (a-histidyl-tRNA synthetase)',
        'Aldosteron': 'Aldosterone',
        'CDT': 'Carbohydrate deficient Transferrin (CDT)',
        'ENA-Screening': 'anti-Extractable nuclear Antigens (ENA-Screen)',
        'DNS Autoantikörper (ds-DNS)': 'anti-double-stranded DNA (dsDNA)',
        'Cardiolipin IgM AK': 'anti-Cardiolipin (CL) IgM',
        'Cardiolipin IgG AK': 'anti-Cardiolipin (CL) IgG',
        'Beta hCG': 'Beta-human chorionic gonadotropine (beta-hCG)',
        'HCV-RNA': 'Hepatitis C Virus ribonucleic acid (HCV-RNA)',
        'HBV-DNA': 'Hepatitis B Virus deoxyribonucleic acid (HBV-DNA)',
        'Nukleosomen Autoantikörper': 'anti-Nucleosome',
        'Histon-Autoantikörper': 'anti-Histone',
        'SmD-Autoantikörper': 'anti-Smith (Sm)',
        'Fibrinogen (funkt.)': 'Fibrinogen (funkt.)',
        'Vitamin D, 1,25 OH': '1,25-Dihydroxy-Vitamin D3 (Calcitriol)',
        'Vitamin D, 25 OH': '25-Hydroxy-Vitamin D3 (Calcidiol)',
        'Vitamin B 6': 'Vitamin B6',
        'DNS Autoantikörper (Crithidien-IFT)': 'anti-double-stranded DNA (Chrithidien-IFT)',
        'AFP (Alpha Fetoprotein)': 'Alpha-1-Fetoprotein (AFP)',
        'Paracetamol': 'sIgE Paracetamol',
        'Triglyceride': 'Triglycerides',
        'Centromer-B-Autoantikörper': 'anti-Centromere Protein B (CENP-B)',
        'Centromer-B-AK': 'anti-Centromere Protein B (CENP-B)',
        'Endomysium IgA-AK': 'anti-Endomysium Antibodies (EMA) IgA',
        'anti-HCV': 'anti-Hepatitis C Virus (HCV)',
        'Insulin Autoantikörper': 'anti-Insulin (IAA)',
        'Cystatin C': 'Cystatin C',
        'Erythropoetin': 'Erythropoetin',
        'Calprotectin i. Stuhl': 'Calprotectin',
        'AAK g. glomeruläre Basalmembran': 'i-Glomerular Basement Membrane (GBM)',
        'p-ANCA (perinukleär)': 'p-anti-Neutrophil Cytoplasmic Antibodies (perinuclear p-ANCA)',
        'p-ANCA-Titer': 'p-anti-Neutrophil Cytoplasmic Antibodies (perinuclear p-ANCA)',
        'Myeloperoxidase-AK (MPO-AK)': 'anti-Myeloperoxidase (MPO)',
        'Acetylcholinrezeptor-AK': 'anti-Acetylcholine Receptor (AChR)',
        'Anti-ds-DNA-Titer': 'anti-double-stranded DNA (Titer)',
        'Anti-ds-DNA': 'anti-double-stranded DNA (dsDNA)',
        'SS-A/Ro60-AK': 'anti-SS-A 60 kDa (Ro 60)',
        'SS-A/Ro52-AK': 'anti-SS-A 52 kDa (Ro 52)',
        'ADAMTS-13-Antikörper': 'anti-ADAMTS-13',
        'ADAMTS-13-Aktivität': 'ADAMTS-13 Activity',
        'ADAMTS-13-Antigen': 'ADAMTS-13 Antigen',
        'Titin-AK i. S.': 'anti-Titin',
        'mikrosomale TPO AK': 'anti-Thyroid peroxidase (aTPO / MAK)',
        'Nukleosomen-AK': 'anti-Nucleosome',
        'CDT nach IFCC': 'Carbohydrate deficient Transferrin (CDT)',
        'Insulin human AAK (IAA)': 'anti-Insulin (IAA)',
        'Zink-Transporter 8 AAK': 'anti-Zinc transporter 8',
        'Humane Anti-Maus-AK': 'Human anti-mouse Antibodies (HAMA)',
        'Histon-AK': 'anti-Histone',
        'DFS70-AK': 'anti-DFS70',
        'HBsAg (Roche)': 'Hepatitis B Virus surface Antigen (HBsAg)',
        'HBsAg  (Roche)': 'Hepatitis B Virus surface Antigen (HBsAg)',
        'HBsAg, quantitativ (Roche)': 'Hepatitis B Virus surface Antigen (HBsAg)',
        'PCNA-AK': 'anti-Proliferating cell nuclear antigen (PCNA)',
        'GAD-AK': 'anti-Glutamate Decarboxylase (GAD)',
        'Inselzell Autoantikörper (ICA)': 'anti-Islet Cell Antibodies (ICA)',
        'Leptospiren IgG': 'anti-Leptospira IgG',
        'Leptospiren IgM': 'anti-Leptospira IgM',
        'M. pneumoniae IgG-AK': 'anti-Mycoplasma pneumoniae IgG',
        'M. pneumoniae IgA-AK': 'anti-Mycoplasma pneumoniae IgA',
        'M. pneumoniae IgM-AK': 'anti-Mycoplasma pneumoniae IgM',
        'C. pneumoniae IgG-AK': 'anti-Chlamydia pneumoniae IgG',
        'C. pneumoniae IgA-AK': 'anti-Chlamydia pneumoniae IgA',
        'C. pneumoniae IgM-AK': 'anti-Chlamydia pneumoniae IgM',
        'C. trachomatis IgG-AK': 'anti-Chlamydia trachomatis IgG',
        'C. trachomatis IgA-AK': 'anti-Chlamydia trachomatis IgA',
        'C. trachomatis IgM-AK': 'anti-Chlamydia trachomatis IgM',
        'Glukose /Plasma': 'Glucose',
        'Glukose': 'Glucose',
        'Bilirubin, ges.': 'Bilirubin total',
        'anorg. Phosphat': 'Phosphat',
        'Helicobacter pylori Antigen quantitativ': 'Helicobacter pylori Antigen'
        }

    parameter_de = parameter_final.copy()

    # Felcode-Liste erstellen
    parameter_codes = new_list(parameter_final, '')
    parameter_codes_excel = open_dm_file_sheet('Aktueller Laborkatalog', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen',
                                              'BioARCHIVE')
    liste_para = list(parameter_codes_excel['Title'])
    liste_para = [para for para in liste_para if str(para) != 'nan']
    
    # Allergie-Parameter-Liste
    liste_allergie = []
    for i in range(len(liste_para)):
        if '(' in liste_para[i]:
            liste_allergie.append(liste_para[i])
    
    # Allergie-Parameter aus Laborkatalog entnehmen
    for i in range(len(parameter_final)):
        if '(' in parameter_final[i]:
            # Allergie-Index entnehmen
            allergie_index = '(' + parameter_final[i].split('(')[-1].split(')')[0] + ')'
            for allergie in liste_allergie:
                if allergie_index in allergie:
                    parameter_final[i] = allergie

    for i in range(len(parameter_final)):
        if parameter_final[i] in para_en_dict:
            parameter_final[i] = para_en_dict[parameter_final[i]]    

    progress_bar.step(10)
    window.update()
            
    # remarks ins engl
    df = open_dm_file('Zellmuster', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen')
    drop_index = []
    for i in range(len(df)):
        if 'AC' not in str(df.iloc[i,0]):
            drop_index.append(i)

    df = df.drop(drop_index).reset_index().drop('index', axis=1)
    remarks_en_dict = {key: value for key, value in zip(df.iloc[:,0], df.iloc[:,1])}
    
    for r in remarks_en_dict:
        for i in range(len(remarks)):
            if r in remarks[i]:
                remarks[i] = remarks_en_dict[r]

    feldcode_para = new_list(parameter_final, '')
    for i in range(len(parameter_final)):
        for j in range(len(liste_para)):
            if parameter_final[i] == liste_para[j]:
                feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                break
        if feldcode_para[i] == '':
            for j in range(len(liste_para)):
                if parameter_final[i].lower() in liste_para[j].lower():
                    feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                    break
            if '-Titer' in parameter_final[i]:
                for j in range(len(liste_para)):
                    if parameter_final[i].replace('-Titer', '') in liste_para[j]:
                        feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                        break
                    try:
                        if para_en_dict[parameter_de[i].replace('-Titer', '')] in liste_para[j]:
                            feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                            break
                    except:
                        pass
                    # ANA 2 (Nummerierung)
                    try:
                        if 'ANA 2' == parameter_de[i].replace('-Titer', ''):
                            parameter_final[i] = 'Antinuclear Antibodies (ANA-Titer)'
                            feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                    except:
                        pass
        if feldcode_para[i] == '':
            # Wort splitten (viele Abkürzungen stehen außerhalb oder in Klammern)
            if ' (' in parameter_final[i]:
                str1 = parameter_final[i].split(' (')[0]
                for j in range(len(liste_para)):
                    if str1 in liste_para[j]:
                        feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                        break
                        
    # measuring method code identifizieren
    method_code = new_list(method_final, '')
    methods_codes_excel = open_dm_file('Methoden Feldcodes', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen')
    if methods_codes_excel.iloc[5,0] == '06=Immunoblot':
        methods_codes_excel.iloc[5,0] = '06=IB (ImmunoBlot)'
    codes_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[0]))
    methods_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[-1]))

    for i in range(len(method_final)):
        for j in range(len(methods_list)):
            if (method_final[i] == methods_list[j][:len(method_final[i])]) | (method_final[i] == methods_list[j][:len(method_final[i])].upper()):
                method_code[i] = codes_list[j]
                method_final[i] = methods_list[j]
                break
    # Infos zu system, manufacturer etc. aus der enstprechenden Excel extrahieren
    excel_info = open_dm_file_sheet('Messsysteme und Preise aller Auftragslabore',
                                    'O:/Datenmanagement/Befunde und Messsysteme/Messsysteme/1_Auftragslabore',
                                    'Labor 28')
    excel_info.columns = excel_info.iloc[1].tolist()
    excel_info = excel_info[['Parameter', 'Angebots- Preis \n(excl. MwSt)', 'Methode', 'Gerät', 'Hersteller/ Gerät', 'KIT', 'Hersteller/ KIT']]
    excel_info = excel_info.iloc[2:]
    excel_info = excel_info.iloc[::-1].reset_index().drop('index', axis=1)
    

    # entsprechende Zeile anhand der Parameternamen entnehmen (zuerst auf deu, dann auf engl)
    index_excel = new_list(parameter_final, '')

    excel_para_list = list(excel_info['Parameter'])

    for i in range(len(parameter_de)):
        if len(parameter_de[i]) > 3:
            for para in excel_para_list[::-1]:
                if parameter_de[i] in para:
                    index_excel[i] = excel_para_list.index(para)
                    break

    for i in range(len(parameter_final)):
        if index_excel[i] == '':
            for para in excel_para_list[::-1]:
                if parameter_final[i] in para:
                    index_excel[i] = excel_para_list.index(para)
                    break
        # falls Parameter in der Messsystem-Tabelle nicht erkannt wurde,
        # kann es auch daran liegen, dass die Parameternamen stärker voneinander abweichen
        if index_excel[i] == '':
            parameter_split = parameter_final[i].split('anti-')
            if '' in parameter_split:
                parameter_split.remove('')
            parameter_split = parameter_split[0]     
            for para_excel in excel_para_list[::-1]:
                para_count = 0
                for para in parameter_split:
                    if para in para_excel:
                        para_count += 1
                if para_count == len(parameter_split):
                    index_excel[i] = excel_para_list.index(para_excel)
                    break

    # Ausnahmen einigen Parametern wegen uneinheitlicher Schreibweise wie TSH, ANA, aTPO, Allergene, Igs
    for i in range(len(parameter_de)):
        if index_excel[i] == '':
            if parameter_de[i] == 'TSH (sensitiv)':
                for para in excel_para_list[::-1]:
                    if ('TSH' in para) & ('TRAK' not in para):
                        index_excel[i] = excel_para_list.index(para)
                        break
            elif 'ANA' in parameter_de[i]:
                for para in excel_para_list[::-1]:
                    if ('ANA' in para) & ('Antinukleäre AK' in para):
                        index_excel[i] = excel_para_list.index(para)
                        break
            elif 'TPO AK' in parameter_de[i]:
                for para in excel_para_list[::-1]:
                    if 'a-TPO' in para:
                        index_excel[i] = excel_para_list.index(para)
                        break
            elif 'sIgE' in parameter_final[i]:
                index_excel[i] = excel_para_list.index('IgE spezifisch (spezifische Allergene)')
            elif parameter_de[i][:2] == 'Ig':
                for para in excel_para_list[::-1]:
                    if parameter_de[i] + ' gesamt' in para:
                        index_excel[i] = excel_para_list.index(para)
                        break                    

    # bei engl. Parameternamen wird nach String innerhalb der Klammern gesucht
    for i in range(len(parameter_final)):
        if index_excel[i] == '':
            if '(' in parameter_final[i]:
                para_suche = parameter_final[i].split('(')[-1].split(')')[0]
                for para in excel_para_list[::-1]:
                    if para_suche in para:
                        index_excel[i] = excel_para_list.index(para)
                        break

    # Die restlichen Infos ergänzen
    system = new_list(parameter_final, 'not specified')
    manu_system = new_list(parameter_final, 'not specified')
    kit = new_list(parameter_final, 'not specified')
    manu_kit = new_list(parameter_final, 'not specified')
    preis = new_list(parameter_final, '')

    progress_bar.step(10)
    window.update()
    
    # Dictionary: Methodenname aus "Messsystem und Preise" und aus "Methoden Feldcode"
    dict_methodname = {
        'Turbidimetrischer Immunoassay': 'TIA (Turbidimetric Immuno Assay)',
        'nephelometrisch': 'Nephelometry',
        'Photometrischer Test': 'Photometry',
        'Enzymatischer Farbtest': 'eFARBT (enzymatic colour test)',
        'Kapillarelektrophorese': 'CE (Capillary Electrophoresis)',
        'Westernblot': 'Western Blot',
        'Atomabsorptionsspektroskopie': 'AAS (Atomic Absorption Spectrometry)',
        'Turbidimetrie': 'Turbidimetry',
        'immunturbidimetrischer Latextest': 'Latex enhanced immunological Turbidimetry',
        'Chemilumineszenz': 'Chemiluminescence'
    }

    for i in range(len(index_excel)):
        if index_excel[i] != '':
            # Methode: Deutsch in Englisch übersetzen
            if str(excel_info['Methode'][index_excel[i]]) in dict_methodname:
                excel_info['Methode'][index_excel[i]] = dict_methodname[excel_info['Methode'][index_excel[i]]]            
            if (str(excel_info['Methode'][index_excel[i]]) in str(method_final[i])) | (str(excel_info['Methode'][index_excel[i]]).split('-')[0] in str(method_final[i])) | (str(method_final[i]).split('(')[0].strip() in str(excel_info['Methode'][index_excel[i]])):            
                if str(excel_info['Gerät'][index_excel[i]]) != 'nan':
                    system[i] = excel_info['Gerät'][index_excel[i]]
                if str(excel_info['Hersteller/ Gerät'][index_excel[i]]) != 'nan':
                    manu_system[i] = excel_info['Hersteller/ Gerät'][index_excel[i]]
                if str(excel_info['KIT'][index_excel[i]]) != 'nan':
                    kit[i] = excel_info['KIT'][index_excel[i]]
                if str(excel_info['Hersteller/ KIT'][index_excel[i]]) != 'nan':
                    manu_kit[i] = excel_info['Hersteller/ KIT'][index_excel[i]]
                if str(excel_info['Angebots- Preis \n(excl. MwSt)'][index_excel[i]]) != 'nan':
                    p = excel_info['Angebots- Preis \n(excl. MwSt)'][index_excel[i]]
                    p = price(p)
                    preis[i] = p
                    
    # DataFrame erstellen
    df = pd.DataFrame({
        'Sample ID': sample_final,
        'Parameter': parameter_final,
        'Feldcode Parameter': feldcode_para,
        'measured value': measured_value,
        'remarks value': remarks,
        'unit': units,
        'reference range': reference_range,
        'unit reference range': units,
        'remarks to reference': remarks_to_reference,
        'measuring method': method_final,
        'measuring methode code':method_code,
        'measuring system': system,
        'manufacturer system': manu_system,
        'measuring kit': kit,
        'manufacturer kit': manu_kit,
        'results taken from': new_list(parameter_final, '01'),
        'gemessen von': new_list(parameter_final, 'Labor 28'),
        'Datum': datum_final,
        'Preise':preis
                      })
    
    # Zeilen mit ANA als Parameter bearbeiten
    drop_index = []
    for i in range(len(df)-1):
        if df.iloc[i,0] == df.iloc[i+1,0]:
            if ('ANA' in df.iloc[i,1]) & ('Titer' not in df.iloc[i,1]) & ('ANA-Titer' in df.iloc[i+1,1]):
                if df.iloc[i,3] == 'positive':
                    if (df.iloc[i,4] != '') & (df.iloc[i+1,4] != ''):
                        df.iloc[i+1,4] = df.iloc[i, 4]+', '+df.iloc[i+1, 4]
                    elif len(df.iloc[i+1,4]) == 0:
                        df.iloc[i+1,4] = df.iloc[i,4]
                    drop_index.append(i)
    
    # Zeilen bearbeiten, wo Parameter doppelt qualitativ und quantitativ vermessen wurden
    for i in range(len(df)-1):
        if (df.iloc[i,0] == df.iloc[i+1,0]) & (df.iloc[i,1] == df.iloc[i+1,1]):    
            if (df.iloc[i,3] == 'positive') | (df.iloc[i, 3] == 'negative'):
                df.iloc[i+1,4] = df.iloc[i,3]
                df.iloc[i+1,6] = df.iloc[i,6]
                drop_index.append(i)
                
    df = df.drop(drop_index).reset_index().drop('index', axis=1)
    
    # in Excel-Tabelle einfügen
    add_to_befunddokumentation(df)
    
    progress_bar.stop()
    progress_bar.grid_forget()
                
    # User Interface
    done = Label(window, text = "\nFertig! #TeamDodo <3",
                   bg = '#eeeee4', font=('Ink free',11))
    done.grid(row=14, column=1, columnspan=2)
    
    
        
######################################### LADR #####################################################
def ladr():
    
    global file_path, done    
    
    try:
        done.grid_forget()
    except:
        pass
    
    # Ladebalken
    space = Label(text='')
    space.grid(row=13, column=1,)
    progress_bar = ttk.Progressbar(window, orient="horizontal", length=250, mode="determinate")
    progress_bar.grid(row=14, column=1, columnspan=5) 
        
    # pdf auswählen
    file_path = filedialog.askopenfilenames(title = "Datei auswählen (LADR)")
    files = list(file_path)
    
    if files == []:
        progress_bar.grid_forget()
        return()
    
    progress_bar.start()
    
    text_list = []
    for file in files:            
        pdfFileObj = open(file,'rb')
        pdfReader = PyPDF2.PdfReader(pdfFileObj)

        totalpages = len(pdfReader.pages)

        for i in range(totalpages):
            pageObj = pdfReader.pages[i]
            text = pageObj.extract_text()
            text_list.append(text)

    for i in range(len(text_list)):
        text_list[i] = text_list[i].split('\n')
        
    progress_bar.step(20)
    window.update()
        
    # aus allen pdf-Befunden eine große pdf erstellen
    # Create a PDF merger object
    pdf_merger = PyPDF2.PdfMerger()
    for pdf_file in files:
        with open(pdf_file, 'rb') as pdf:
            pdf_merger.append(pdf)
    output_pdf = 'ladr_{}.pdf'.format(heute)
    # Write the merged PDF to the output file
    with open(output_pdf, 'wb') as output:
        pdf_merger.write(output)
    # Close the merger
    pdf_merger.close()
    
    progress_bar.step(10)
    window.update()
        
    # sample ID ablesen
    sample_id = new_list(text_list, '')
    datum = new_list(text_list, '')
    for i in range(len(text_list)):
        for j in range(len(text_list[i])):
            if 'Name' in text_list[i][j]:
                sample_id[i] = text_list[i][j].split('Name')[-1].split('Eingang')[0].strip()[:-3]
                break
        # Datum ablesen
        for j in range(len(text_list[i])):
            if 'Eingang ' in text_list[i][j]:
                datum[i] = text_list[i][j].split('Eingang')[-1].strip()
                
    # text_list bearbeiten, sodass text_list nur noch Parameterdaten enthält
    for i in range(len(text_list)):
        for j in range(len(text_list[i])):
            if 'Referenzbereich' in text_list[i][j]:
                start_index = j+1
        text_list[i] = text_list[i][start_index:]
    # Zeilen mit Material und Abnahme aus text_list entfernen
    for i in range(len(text_list)):
        drop_index = []
        for j in range(len(text_list[i])):
            if ('Material' in text_list[i][j]) | ('Abnahme' in text_list[i][j]) | ('Seite' in text_list[i][j]):
                drop_index.append(j)
        text_list[i] = [text_list[i][j] for j in range(len(text_list[i])) if j not in drop_index]
    
        # manchmal steht der Referenzbereich erst in der nächsten Zeile
        # und es wird nach Alter + Geschlecht unterschieden
        # Referenzbereich aus erster Zeile entnehmen und restl. löschen
        drop_index = []
        for j in range(len(text_list[i])):
            if text_list[i][j][len(text_list[i][j])-len('Erwachsene:'):] == 'Erwachsene:':
                if 'weiblich:' in text_list[i][j+1]:
                    text_list[i][j] = text_list[i][j] + text_list[i][j+1].split('weiblich:')[-1]
                    drop_index.append(j+1)
                elif '>' == text_list[i][j+1][0]:
                    text_list[i][j] = text_list[i][j] + text_list[i][j+1][1:]
                    drop_index.append(j+1)
        text_list[i] = [text_list[i][j] for j in range(len(text_list[i])) if j not in drop_index]
        # alle restl. Zeilen entfernen
        drop_index = []
        for j in range(len(text_list[i])):
            if 'männlich:' in text_list[i][j]:
                drop_index.append(j)
            if '>' == text_list[i][j][0]:
                drop_index.append(j)
        text_list[i] = [text_list[i][j] for j in range(len(text_list[i])) if j not in drop_index]
                    
    progress_bar.step(20)
    window.update()
            
    paras = new_list(text_list, '')
    measured_values = new_list(text_list, '')
    units = new_list(text_list, '')
    references = new_list(text_list, '')
        
    zeichen = ['> ', '< ', '=>', '=<']
    pos_neg = ['positiv', 'negativ']

    for i in range(len(text_list)):
        para = []
        measured_value = []
        unit = []
        reference = []

        for j in range(len(text_list[i])):
            for char in text_list[i][j]:
                if char.isdigit():
                    parameter = text_list[i][j].split(char)[0].strip()
                    para.append(parameter)
                    text_list[i][j] = text_list[i][j].split(parameter)[-1].strip()
                    val = text_list[i][j].split(' ')[0]
                    measured_value.append(val)
                    text_list[i][j] = text_list[i][j][len(val):].strip()
                    un = text_list[i][j].split(' ')[0]
                    unit.append(un)
                    ref = text_list[i][j].split(un)[-1].strip()
                    if ':' in ref:
                        ref = ref.split(':')[-1].strip()
                    reference.append(ref)
                    break

        paras[i] = para
        measured_values[i] = measured_value
        units[i] = unit
        references[i] = reference
    
    # Messwert und Referenzbereich IDA-konform umwandeln
    for i in range(len(measured_values)):
        for j in range(len(measured_values[i])):
            measured_values[i][j] = decimal_value(measured_values[i][j])
            if '-' in references[i][j]:
                first_value = decimal_value(references[i][j].split('-')[0].strip())
                second_value = decimal_value(references[i][j].split('-')[-1].strip())
                references[i][j] = first_value + ' - ' + second_value
            else:
                if references[i][j] != '':
                    try:
                        references[i][j] = decimal_value(references[i][j])
                    except:
                        for zeichen in ['<', '>']:
                            if zeichen in references[i][j]:
                                value = references[i][j].split(zeichen)[-1].strip()
                                value = decimal_value(value)
                                references[i][j] = zeichen + ' ' + value
    
    # sample_id, paras, datum auf dieselbe Dimension bringen
    multi_list = []
    for i in range(len(paras)):
        multi_list.append(len(paras[i]))

    sample_final = []
    datum_final = []

    for x,y in zip(sample_id, multi_list):
        sample_final.extend([x]*y)
    for x,y in zip(datum, multi_list):
        datum_final.extend([x]*y)

    progress_bar.step(10)
    window.update()
        
    paras = [item for sublist in paras for item in sublist]
    measured_values = [item for sublist in measured_values for item in sublist]
    references = [item for sublist in references for item in sublist]
    units = [item for sublist in units for item in sublist]
    
    # Parameter ins engl übersetzen
    para_en_dict = {
        'thrombozyten': 'Thrombocytes',
        'leukozyten': 'Leucocytes',
        'erythrozyten': 'Erythrocytes',
        'gesamt eiweiss': 'Total Protein',
        'igg gesamt': 'Immunoglobulin G total (total IgG)',
        'ige gesamt': 'Immunoglobulin E total (total IgE)',
        'haemoglobin': 'Hemoglobin (Hb)',
        'haematokrit': 'Hematokrit',
        'mch': 'Mean Corpuscular Hemoglobin (MCH, HbE)',
        'mchc': 'Mean Corpuscular Hemoglobin Concentration (MCHC)',
        'mcv': 'Mean Corpuscular Volume (MCV)',
        'rdwcv': 'Red Blood Cell Distribution Width (RDW)',
        'carb defic transferr': 'Carbohydrate deficient Transferrin (CDT)'
    }
    for i in range(len(paras)):
        for entry_para in para_en_dict:
            if entry_para in paras[i].lower():
                paras[i] = para_en_dict[entry_para]

    # Felcode-Liste erstellen
    parameter_codes = new_list(paras, '')
    parameter_codes_excel = open_dm_file_sheet('Aktueller Laborkatalog', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen',
                                              'BioARCHIVE')
    liste_para = list(parameter_codes_excel['Title'])
    liste_para = [para for para in liste_para if str(para) != 'nan']

    feldcode_para = new_list(paras, '')
    for i in range(len(paras)):
        for j in range(len(liste_para)):
            if paras[i] == liste_para[j].strip():
                feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                break    

    # restl. Infos aus der enstprechenden Excel extrahieren
    excel_info = open_dm_file_sheet('Messsysteme und Preise aller Auftragslabore', 'O:/Datenmanagement/Befunde und Messsysteme/Messsysteme/1_Auftragslabore',
                                    'LADR (ehem. HLBB)')
    excel_info.columns = excel_info.iloc[2]
    excel_info = excel_info[['laboratory parameter', 'Angebots- Preis \n(excl. MwSt)', 'measuring method', 'measuring System', 'manufacturer of measuring system', 'measuring kit', 'manufacturer of measuring kit']]
    excel_info = excel_info.iloc[3:]
    excel_info = excel_info.iloc[::-1].reset_index().drop('index', axis=1)

    # entsprechende Zeile anhand der Parameternamen entnehmen
    index_excel = new_list(paras, '')
    excel_para_list = list(excel_info['laboratory parameter'])
    
    progress_bar.step(10)
    window.update()

    for i in range(len(paras)):
        for para in excel_para_list:
            if paras[i] in para:
                index_excel[i] = excel_para_list.index(para)
                break
            # Suche nach Wort in Klammern
            if index_excel[i] == '':
                if '(' in paras[i]:
                    if paras[i].split('(')[-1].split(')')[0] in para:
                        index_excel[i] = excel_para_list.index(para)
                        break                   

    # Infos ergänzen
    method = new_list(paras, 'not specified')
    system = new_list(paras, 'not specified')
    manu_system = new_list(paras, 'not specified')
    kit = new_list(paras, 'not specified')
    manu_kit = new_list(paras, 'not specified')
    preis = new_list(paras, '')

    for i in range(len(index_excel)):
        if index_excel[i] != '':
            if str(excel_info['measuring method'][index_excel[i]]) != 'nan':
                method[i] = excel_info['measuring method'][index_excel[i]]        
            if str(excel_info['measuring System'][index_excel[i]]) != 'nan':
                system[i] = excel_info['measuring System'][index_excel[i]]
            if str(excel_info['manufacturer of measuring system'][index_excel[i]]) != 'nan':
                manu_system[i] = excel_info['manufacturer of measuring system'][index_excel[i]]
            if str(excel_info['measuring kit'][index_excel[i]]) != 'nan':
                kit[i] = excel_info['measuring kit'][index_excel[i]]
            if str(excel_info['manufacturer of measuring kit'][index_excel[i]]) != 'nan':
                manu_kit[i] = excel_info['manufacturer of measuring kit'][index_excel[i]]
            if str(excel_info['Angebots- Preis \n(excl. MwSt)'][index_excel[i]]) != 'nan':
                p = excel_info['Angebots- Preis \n(excl. MwSt)'][index_excel[i]]
                p = float(p)
                p = "{:.2f}".format(p).replace('.', ',')
                preis[i] = p

    # method_code-Liste erstellen
    method_code = new_list(method, '')
    methods_codes_excel = open_dm_file('Methoden Feldcodes', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen')
    if methods_codes_excel.iloc[5,0] == '06=Immunoblot':
        methods_codes_excel.iloc[5,0] = '06=IB (ImmunoBlot)'
    codes_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[0]))
    methods_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[-1]))

    for i in range(len(method)):
        for j in range(len(methods_list)):
            if method[i] == methods_list[j][:len(method[i])]:
                method_code[i] = codes_list[j]
                break
                
    progress_bar.step(20)
    window.update()
    
    # DataFrame erstellen
    df = pd.DataFrame({
        'Sample ID': sample_final,
        'Parameter': paras,
        'Feldcode Parameter': feldcode_para,
        'measured value': measured_values,
        'remarks value': new_list(paras, ''),
        'unit': units,
        'reference range': references,
        'unit reference range': units,
        'remarks to reference': new_list(paras, ''),
        'measuring method': method,
        'measuring methode code':method_code,
        'measuring system': system,
        'manufacturer system': manu_system,
        'measuring kit': kit,
        'manufacturer kit': manu_kit,
        'results taken from': new_list(paras, '01'),
        'gemessen von': new_list(paras, 'LADR Hennigsdorf'),
        'Datum': datum_final,
        'Preise':preis
        })
    
    # df bearbeiten: doppelte Info rauslöschen   
    add_to_befunddokumentation(df)
    
    progress_bar.stop()
    progress_bar.grid_forget()
    
    done = Label(window, text = "\nFertig! #TeamDodo <3",
                   bg = '#eeeee4', font=('Ink free',11))
    done.grid(row=14, column=1, columnspan=2)
    
    
########################################## Synlab ###################################################   

def synlab():
    
    global file_path, done    
    
    try:
        done.grid_forget()
    except:
        pass
    
    # Ladebalken
    space = Label(text='')
    space.grid(row=13, column=1,)
    progress_bar = ttk.Progressbar(window, orient="horizontal", length=250, mode="determinate")
    progress_bar.grid(row=14, column=1, columnspan=5) 
    
    # pdf auswählen
    file_path = filedialog.askopenfilename(title = "Datei auswählen (Synlab)")
    
    if file_path == '':
        progress_bar.grid_forget()
        return()
    
    pages = convert_from_path(file_path, poppler_path = r'wichtig\poppler-0.68.0\bin')
    pytesseract.pytesseract.tesseract_cmd = r'wichtig\Tesseract-OCR\tesseract.exe'
    custom_config = r'--oem 3 --psm 6'
    
    progress_bar.start()

    def im_to_text(page):
        image = np.array(page)
        im = Image.fromarray(image)
        details = pytesseract.image_to_data(im, output_type=Output.DICT, config=custom_config)
        total_boxes = len(details['text'])
        for sequence_number in range(total_boxes):
            if float(details['conf'][sequence_number]) > 30:
                (x, y, w, h) = (details['left'][sequence_number], details['top'][sequence_number], details['width'][sequence_number],  details['height'][sequence_number])

        parse_text = []
        word_list = []
        last_word = ''
        for word in details['text']:
            if word!='':
                word_list.append(word)
                last_word = word
            if (last_word!='' and word == '') or (word==details['text'][-1]):
                parse_text.append(word_list)
                word_list = []

        for i in range(len(parse_text)):
            parse_text[i] = ' '.join(parse_text[i])
        return(parse_text)

    pages_bag = db.from_sequence(pages)
    result = pages_bag.map(im_to_text)

    text_list = result.compute()
    
    progress_bar.step(20)
    window.update()

    # Deckblatt rausnehmen
    drop_index = []
    for i in range(len(text_list)):
        weiter = False
        for j in range(len(text_list[i])):
            if 'Received' in text_list[i][j]:
                weiter = True
                break
        if weiter:
            continue
        drop_index.append(i)
    if drop_index != []:
        text_list = [el for i, el in enumerate(text_list) if i not in drop_index]
    # Seiten mit IP-Nummer rausnehmen
    drop_index = []
    for i in range(len(text_list)):
        for j in range(len(text_list[i])):
            if (('Received' in text_list[i][j]) & ('IP' in text_list[i][j])) | (('Received' in text_list[i][j]) & ('1P' in text_list[i][j])):
                drop_index.append(i)
                break
    if drop_index != []:
        text_list = [el for i, el in enumerate(text_list) if i not in drop_index]
    
    # sample ID und Datum ablesen, diese sind generell immer in derselben Spalte
    sample_id = new_list(text_list, '')
    datum = new_list(text_list, '')
    for i in range(len(text_list)):
        for j in range(len(text_list[i])):
            if 'Received' in text_list[i][j]:
                row_info = text_list[i][j].split('Received')
                sample = row_info[0].strip().split(' ')
                for s in sample:
                    if len(s) > 5:
                        sample = s
                sample_id[i] = 'I' + sample[1:-3]
                # manchmal wird beim Einlesen statt 0 eine 6 eingelesen, Korrektur:
                if '6' in sample_id[i][:4]:
                    sample_id[i] = sample_id[i][:4].replace('6', '0') + sample_id[i][4:]
            if 'Printed' in text_list[i][j]:
                date = text_list[i][j].split(' ')[-1]
                datum[i] = date[:-2] + '20' + date[-2:]
                # Infos über Datum-Zeile aus text_list löschen
                text_list[i] = text_list[i][j+1:]
                break
                
    progress_bar.step(10)
    window.update()
            
    # weitere unnötige Zeilen aus text_list löschen
    for i in range(len(text_list)):
        drop_index = []
        for j in range(len(text_list[i])):
            if len(text_list[i][j]) < 5:
                drop_index.append(j)
        text_list[i] = [el for l, el in enumerate(text_list[i]) if l not in drop_index]
    
    # Zeile mit 'Verdünnung' finden
    for i in range(len(text_list)):
        for j in range(1,len(text_list[i])):
            if 'Verd.' in text_list[i][j]:
                text_list[i][j-1] = text_list[i][j-1] + ' ' + text_list[i][j]
            # manchmal können bei länger geschriebenen Wörtern Schreibfehler auftreten
            for v in ['Verdünnung', 'Dilution']:
                match = regex.search(r'({}){}'.format(v, '{e<3}'), text_list[i][j])
                if match:
                    verd_text = text_list[i][j].split(match.group())[-1].strip()
                    text_list[i][j-1] = text_list[i][j-1] + ' ' + v + ' ' + verd_text
                    
    # Zeilen mit den Parametern und Messwerten finden
    parameter = new_list(text_list, '')
    parameter_final = new_list(text_list, '')
    value_zeichen = ['<', '>', 'positive', 'negative','reactive', 'non-reac tive', 'non-reac ', 'non-reactive']

    progress_bar.step(10)
    window.update()
    
    # jede Parameter-Zeile beginnt mit 4 Ziffern
    # Liste mit den Parameternamen erstellen
    muster = r'\d{4}'
    for i in range(len(text_list)):
        para = []
        for j in range(len(text_list[i])):
            match = re.search(muster, text_list[i][j][:4])
            if match:
                para.append(text_list[i][j][8:].strip())
        parameter[i] = para
    for i in range(len(parameter)):
        para = []
        for j in range(len(parameter[i])):
            para_final = ''
            for v in value_zeichen:
                if v in parameter[i][j]:
                    para_final = parameter[i][j].split(v)[0].strip()
                    break
            if para_final == '':
                muster = r' \d{1}'
                match = re.search(muster, parameter[i][j])
                if match:
                    para_final = parameter[i][j][:match.span()[0]].strip()
            # manchmal wird statt T eine 1 eingelesen
            if ('free' in para_final) & ('1' in para_final):
                para_final = para_final.replace('1', 'T')
            para.append(para_final)
        parameter_final[i] = para

    # Parameter-Liste bearbeiten, um Messwerte ablesen zu können
    for i in range(len(parameter)):
        for j in range(len(parameter[i])):
            parameter[i][j] = parameter[i][j][len(parameter_final[i][j]):].strip()

    # Messwerte ablesen
    measured_value = new_list(parameter, '')
    for i in range(len(measured_value)):
        measured_value[i] = new_list(parameter[i], '')

    non_reactive = ['non-reac tive', 'non-reac ']
    neg_pos = ['negative', 'positive']
    zeichen = ['< ', '> ', '<', '>']
    for i in range(len(parameter)):
        for j in range(len(parameter[i])):
            for non in non_reactive:
                if parameter[i][j][:len(non)] == non:
                    measured_value[i][j] = 'non reactive'
                    parameter[i][j] = parameter[i][j][len(non):].strip()
                    break
            for x in neg_pos:
                if x == parameter[i][j][:len(x)]:
                    measured_value[i][j] = x
                    parameter[i][j] = parameter[i][j][len(x):].strip()
                    break
            for z in zeichen:
                if z == parameter[i][j][:len(z)]:
                    value = parameter[i][j][len(z):].split(' ')[0]
                    parameter[i][j] = ' '.join(parameter[i][j][len(z):].split(' ')[1:]).strip()
                    try:
                        value = decimal_value(value)
                    except:
                        pass
                    measured_value[i][j] = z.split(' ')[0] + ' ' + value
                    break
            if measured_value[i][j] == '':
                value = parameter[i][j].split(' ')[0]
                try:
                    value = decimal_value(value)
                except:
                    pass
                measured_value[i][j] = value
                parameter[i][j] = ' '.join(parameter[i][j].split(' ')[1:]).strip()
                
    progress_bar.step(10)
    window.update()
                
    # remarks-Liste erstellen
    remarks = new_list(parameter, '')
    for i in range(len(remarks)):
        remarks[i] = new_list(parameter[i], '')
    for i in range(len(parameter)):
        for j in range(len(parameter[i])):
            for verd in ['Dilution:', 'Verd.', 'Dilution :']:
                if verd in parameter[i][j]:
                    dil_remark = parameter[i][j].split(verd)[-1].strip()                  
                    remarks[i][j] = 'dilution: ' + parameter[i][j].split(verd)[-1].strip()
                    parameter[i][j] = parameter[i][j].split(verd)[0]
                    
            if 'Verdünnung' in parameter[i][j]:
                dil_remark = parameter[i][j].split('Verdünnung')[-2].strip().split(' ')[-1]
                if '-' not in dil_remark:
                    dil_remark = parameter[i][j].split('Verdünnung')[-1].strip().split(' ')[-1]
                remarks[i][j] = 'dilution: ' + dil_remark
                if 'Messung' in parameter[i][j]:
                    parameter[i][j] = parameter[i][j].split('Messung')[0].strip()
                else:
                    parameter[i][j] = ' '.join(parameter[i][j].split('Verdünnung')[0].strip().split(' ')[:-1])

    # unit und reference range ablesen
    unit = new_list(parameter, '-')
    reference = new_list(parameter, '-')
    for i in range(len(unit)):
        unit[i] = new_list(parameter[i], '-')
        reference[i] = new_list(parameter[i], '-')
    for i in range(len(parameter)):
        for j in range(len(parameter[i])):
            
            # fehlerhafte Zeichen entfernen
            if '«' in parameter[i][j]:
                parameter[i][j] = parameter[i][j].replace('«', '')
            
            if len(parameter[i][j].split(' ')) == 1:
                reference[i][j] = parameter[i][j]
            else:
                unit[i][j] = parameter[i][j].split(' ')[0]
                if unit[i][j] == 'N/A':
                    unit[i][j] = '-'
                # falsch abgelesene Zeichen korrigieren
                wrong_l = ['1', '[', ']', '|']
                for l in wrong_l:
                    if ('/' in unit[i][j]) & (unit[i][j].split('/')[-1]==l):
                        unit[i][j] = unit[i][j][:-1] + 'l'
                if 'T' in unit[i][j]:
                    unit[i][j] = unit[i][j].replace('T', 'I')
                if 'mL' in unit[i][j]:
                    unit[i][j] = unit[i][j].replace('mL', 'ml')
                # falls keine Unit angegeben wurde, muss Unit über ein dictionary abgelesen werden
                if (unit[i][j] == '0') | ('<' in unit[i][j]):
                    unit[i][j] = '-'
                else:
                    parameter[i][j] = ''.join(parameter[i][j].split(' ')[1:])
                # unit-Korrektur
                if 'Uu/' in unit[i][j]:
                    unit[i][j] = unit[i][j].replace('Uu/', 'U/')
                if 'uU/' in unit[i][j]:
                    unit[i][j] = unit[i][j].replace('uU/', 'U/')
                ref = parameter[i][j]
                # manchmal wird statt 0 ein Q eingelesen, diesen Fehler korrigieren
                if 'Q' in ref:
                    ref = ref.replace('Q', '0')
                
                try:
                    if ('-' in ref) & (ref != 'non-reactive'):
                        ref = ref.split('-')
                        ref1 = ref[0].strip()
                        ref1 = decimal_value(ref1)
                        ref2 = ref[-1].strip().split(' ')[0].strip()
                        ref2 = decimal_value(ref2)
                        ref = ref1 + ' - ' + ref2
                        if '0,00 - ' in ref:
                            ref = ref.replace('0,00 - ', '< ')
                except:
                    pass
                reference[i][j] = ref
            # 'non-reactive' zu 'non reactive' umwandeln
            if reference[i][j] == 'non-reactive':
                reference[i][j] = 'non reactive'
    
    # eindimensionale Listen erstellen
    multi_list = []
    for i in range(len(parameter)):
        multi_list.append(len(parameter[i]))

    sample_final = []
    datum_final = []
    for x,y in zip(sample_id, multi_list):
        sample_final.extend([x]*y)
    for x,y in zip(datum, multi_list):    
        datum_final.extend([x]*y)

    parameter_final = [item for sublist in parameter_final for item in sublist]
    measured_value = [item for sublist in measured_value for item in sublist]
    unit = [item for sublist in unit for item in sublist]
    reference = [item for sublist in reference for item in sublist]
    remarks = [item for sublist in remarks for item in sublist]
    
    progress_bar.step(10)
    window.update()
    
    # Die Parameter werden (sofern es geht) ins Englische übersetzt
    para_en_dict = {
        'HIV, AG/AB': 'anti-Human Immunodeficiency Virus (HIV)',
        'HIV, p17AG/AB': 'anti-Human Immunodeficiency Virus p17 (HIV p17)',
        'HIV, p18AG/AB': 'anti-Human Immunodeficiency Virus p18 (HIV p18)',
        'HIV, p24AG/AB': 'anti-Human Immunodeficiency Virus p24 (HIV p24)',
        'HIV, p31AG/AB': 'anti-Human Immunodeficiency Virus p31 (HIV p31)',
        'HIV, sgp105AG/AB': 'anti-Human Immunodeficiency Virus sgp105 (HIV sgp105)',
        'HIV 1, AG/AB': 'anti-Human Immunodeficiency Virus-1 (HIV-1)',
        'HIV 1, gp120AG/AB': 'anti-Human Immunodeficiency Virus-1 gp120 (HIV-1 gp120)',
        'HIV 1, gp41AG/AB': 'anti-Human Immunodeficiency Virus-1 gp41 (HIV-1 gp41)',
        'HIV 1, p15AG/AB': 'anti-Human Immunodeficiency Virus-1 p15 (HIV-1 p15)',
        'HIV 1, p24AG/AB': 'anti-Human Immunodeficiency Virus-1 p24 (HIV-1 p24)',        
        'HIV 1+2, AG/AB': 'anti-Human Immunodeficiency Virus-1/2 (HIV-1/2)',
        'HIV 1+2, gp105AG/AB': 'anti-Human Immunodeficiency Virus-1/2 gp105 (HIV-1/2 gp105)',
        'HIV 1+2, gp120AG/AB': 'anti-Human Immunodeficiency Virus-1/2 gp120 (HIV-1/2 gp120)',
        'HIV 1+2, gp36AG/AB': 'anti-Human Immunodeficiency Virus-1/2 gp36 (HIV-1/2 gp36)',
        'HIV 1+2, gp41AG/AB': 'anti-Human Immunodeficiency Virus-1/2 gp41 (HIV-1/2 gp41)',
        'HIV 1+2, p17AG/AB': 'anti-Human Immunodeficiency Virus-1/2 p17 (HIV-1/2 p17)',        
        'HIV 1+2, p24AG/AB': 'anti-Human Immunodeficiency Virus-1/2 p24 (HIV-1/2 p24)',
        'HIV 1+2, p31AG/AB': 'anti-Human Immunodeficiency Virus-1/2 p31 (HIV-1/2 p31)',
        'HIV 2, AG/AB': 'anti-Human Immunodeficiency Virus-2 (HIV-2)',
        'HIV 2, gp105AG/AB': 'anti-Human Immunodeficiency Virus-2 gp105 (HIV-2 gp105)',
        'HIV 2, gp36AG/AB': 'anti-Human Immunodeficiency Virus-2 gp36 (HIV-2 gp36)',        
        'Anti-HCV (Hep. C-AB)': 'anti-Hepatitis C Virus (HCV)',
        'HBs-Ag (Hep. B Surf. Ag)': 'Hepatitis B Virus surface Antigen (HBsAg)',
        'AAB Thyroid Microsomes': 'anti-Thyroid peroxidase (aTPO / MAK)',
        'AAB Thyreoglobulin': 'anti-Thyreoglobulin (aTg / TAK)',
        'T3, free': 'fT3',
        'T4, free': 'fT4'
    }
    
    unit_dict = {   
        'AAB Thyroid Microsomes': 'kU/l',
        'AAB Thyreoglobulin': 'kU/l',
        'T3, free': 'pmol/l',
        'T4, free': 'pmol/l'        
    }
    
    # Units ergänzen
    for i in range(len(unit)):
        if unit[i] == '-':
            if parameter_final[i] in unit_dict:
                unit[i] = unit_dict[parameter_final[i]]
    
    parameter_de = parameter_final.copy()
    liste_para_keys = list(para_en_dict.keys())

    check_index = []
    for i in range(len(parameter_final)):
        index_flag = True
        for en_para in liste_para_keys:
            if parameter_final[i] in en_para:
                parameter_final[i] = para_en_dict[en_para]
                index_flag = False
        if index_flag:
            check_index.append(i)
    # Falls Parameter nicht gefunden wurde, weil dieser falsch eingelesen wurde,
    # wird dieser nochmal gecheckt und korrigiert
    if check_index != []:
        para_check = []
        for index in check_index:
            para_check.append(parameter_de[index])
        for i in range(len(para_check)):
            for j in range(len(liste_para_keys)):
                match = regex.search(r'({}){}'.format(para_check[i], '{e<3}'), liste_para_keys[j])
                if match:
                    para_check[i] = para_en_dict[liste_para_keys[j]]
    for i in range(len(check_index)):
        parameter_final[check_index[i]] = para_check[i]   
    
    # Felcode-Liste erstellen
    parameter_codes = new_list(parameter_final, '')
    parameter_codes_excel = open_dm_file_sheet('Aktueller Laborkatalog', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen',
                                              'BioARCHIVE')
    liste_para = list(parameter_codes_excel['Title'])
    liste_para = [para for para in liste_para if str(para) != 'nan']

    feldcode_para = new_list(parameter_final, '')
    for i in range(len(parameter_final)):
        for j in range(len(liste_para)):
            if parameter_final[i] == liste_para[j]:
                feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                break
        if feldcode_para[i] == '':
            for j in range(len(liste_para)):
                if parameter_final[i].lower() in liste_para[j].lower():
                    feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                    break
        if feldcode_para[i] == '':
            # Wort splitten (viele Abkürzungen stehen in Klammern)
            if ' (' in parameter_final[i]:
                str1 = parameter_final[i].split(' (')[1]
                for j in range(len(liste_para)):
                    if str1 in liste_para[j]:
                        feldcode_para[i] = int(parameter_codes_excel.iloc[j,:]['Code'])
                        break
                        
    progress_bar.step(10)
    window.update()
                        
    # restl. Infos aus der enstprechenden Excel extrahieren
    excel_info = open_dm_file_sheet('Messsysteme und Preise aller Auftragslabore', 'O:/Datenmanagement/Befunde und Messsysteme/Messsysteme/1_Auftragslabore',
                                   'Synlab (ehem. W&T)')
    excel_info.columns = excel_info.iloc[1]
    excel_info = excel_info[['Parameter', 'Angebots- Preis \n(excl. MwSt)', 'Methode', 'Gerät', 'Hersteller/ Gerät', 'KIT', 'Hersteller/ KIT']]
    excel_info = excel_info.iloc[2:]
    excel_info = excel_info.iloc[::-1].reset_index().drop('index', axis=1)    

    # entsprechende Zeile anhand der Parameternamen entnehmen (zuerst auf deu, dann auf engl)
    index_excel = new_list(parameter_final, '')
    excel_para_list = list(excel_info['Parameter'])

    # bei engl. Parameternamen wird nach String innerhalb der Klammern gesucht
    for i in range(len(parameter_final)):
        if '(' in parameter_final[i]:
            para_suche = parameter_final[i].split('(')[-1].split(')')[0]
            for para in excel_para_list:
                if para_suche in para:
                    if excel_info.iloc[excel_para_list.index(para),3] == 'siehe Einzelparameter':
                        continue
                    index_excel[i] = excel_para_list.index(para)
                    break
            if index_excel[i] == '':
                # String noch weiter splitten, da innerhalb der Klammer meherere Abkürzungen stehen können
                if '/' in para_suche:
                    para_suche = para_suche.split('/')[-1].strip()
                    for para in excel_para_list:
                        if para_suche in para:
                            index_excel[i] = excel_para_list.index(para)
                            break
                if ' ' in para_suche:
                    para1 = para_suche.split(' ')[0]
                    para2 = para_suche.split(' ')[1]
                    for para in excel_para_list:
                        if (para1 in para) & (para2 in para):
                            index_excel[i] = excel_para_list.index(para)
                            break
                    
        if index_excel[i] == '':
            for para in excel_para_list:
                if parameter_final[i] in para:
                    index_excel[i] = excel_para_list.index(para)
                    break    
    
    # Infos ergänzen
    method = new_list(parameter_final, 'not specified')
    system = new_list(parameter_final, 'not specified')
    manu_system = new_list(parameter_final, 'not specified')
    kit = new_list(parameter_final, 'not specified')
    manu_kit = new_list(parameter_final, 'not specified')
    preis = new_list(parameter_final, '')

    for i in range(len(index_excel)):
        if index_excel[i] != '':
            if str(excel_info['Methode'][index_excel[i]]) != 'nan':
                method[i] = excel_info['Methode'][index_excel[i]]        
            if str(excel_info['Gerät'][index_excel[i]]) != 'nan':
                system[i] = excel_info['Gerät'][index_excel[i]]
            if str(excel_info['Hersteller/ Gerät'][index_excel[i]]) != 'nan':
                manu_system[i] = excel_info['Hersteller/ Gerät'][index_excel[i]]
            if str(excel_info['KIT'][index_excel[i]]) != 'nan':
                kit[i] = excel_info['KIT'][index_excel[i]]
            if str(excel_info['Hersteller/ KIT'][index_excel[i]]) != 'nan':
                manu_kit[i] = excel_info['Hersteller/ KIT'][index_excel[i]]
            if str(excel_info['Angebots- Preis \n(excl. MwSt)'][index_excel[i]]) != 'nan':
                p = excel_info['Angebots- Preis \n(excl. MwSt)'][index_excel[i]]
                p = decimal_value(p)
                preis[i] = p
                
    progress_bar.step(10)
    window.update()

    # method_code-Liste erstellen
    method_code = new_list(method, '')
    methods_codes_excel = open_dm_file('Methoden Feldcodes', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen')
    if methods_codes_excel.iloc[5,0] == '06=Immunoblot':
        methods_codes_excel.iloc[5,0] = '06=IB (ImmunoBlot)'
    codes_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[0]))
    methods_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[-1]))

    for i in range(len(method)):
        for j in range(len(methods_list)):
            if method[i] == methods_list[j][:len(method[i])]:
                method_code[i] = codes_list[j]
                break    

    # DataFrame erstellen
    df = pd.DataFrame({
        'Sample ID': sample_final,
        'Parameter': parameter_final,
        'Feldcode Parameter': feldcode_para,
        'measured value': measured_value,
        'remarks value': remarks,
        'unit': unit,
        'reference range': reference,
        'unit reference range': unit,
        'remarks to reference': new_list(parameter_final, ''),
        'measuring method': method,
        'measuring methode code':method_code,
        'measuring system': system,
        'manufacturer system': manu_system,
        'measuring kit': kit,
        'manufacturer kit': manu_kit,
        'results taken from': new_list(parameter_final, '01'),
        'gemessen von': new_list(parameter_final, 'W&T'),
        'Datum': datum_final,
        'Preise':preis
        })
    
    progress_bar.step(10)
    window.update()
    
    add_to_befunddokumentation(df)
    
    progress_bar.stop()
    progress_bar.grid_forget()
    
    done = Label(window, text = "\nFertig! #TeamDodo <3",
                   bg = '#eeeee4', font=('Ink free',11))
    done.grid(row=14, column=1, columnspan=2)
    

################################### Spranger ##########################################################
def spranger_read():
    # pdf auswählen
    global file_path, back_to_homepage, text_nofiles, csv_file
    
    # Ladebalken
    space = Label(text='')
    space.grid(row=13, column=1,)
    progress_bar = ttk.Progressbar(window, orient="horizontal", length=250, mode="determinate")
    progress_bar.grid(row=14, column=1, columnspan=5)
    
    try:
        text_nofiles.pack_forget()
    except:
        pass
    
    try:
        done.grid_forget()
    except:
        pass
    
    folder_path = r'O:\Versand\Labore\Labor Dr. Spranger\Import'
    #folder_path = "C:/Users/l.nguyen quoc/OneDrive - in.vent Diagnostica GmbH/DM/OCR/Spranger Tests"
    files = glob.glob(f'{folder_path}/*.csv')
    
    progress_bar.start()
    
    if files == []:
        text_nofiles = Label(root2, text = "\nEs sind keine Befunde vorhanden",
                       bg = '#eeeee4', font=('Ink free',11))
        text_nofiles.pack()        
    else:
        datum = ['']*len(files)
        sample_id = ['']*len(files)
        parameter = ['']*len(files)
        measured_value = ['']*len(files)
        for i in range(len(files)):
            # Datum ablesen
            file = files[i]
            datum[i] = file.split('\\')[-1][8:10] + '.' + file.split('\\')[-1][5:7] + '.' + file.split('\\')[-1][:4]
            try:
                csv_file = pd.read_csv(file, sep=';')
            # falls Datei geöffnet ist
            except:
                if '~$' in file:
                    file = path.replace('~$', '')
                # Generate a temporary directory to store the copied file
                temp_dir = tempfile.mkdtemp()        
                # Generate a temporary file name with a .xlsx extension in the temporary directory
                temp_file_path = tempfile.mktemp(suffix='.csv', dir=temp_dir)        
                # Copy the open file to the temporary location
                shutil.copy2(file, temp_file_path)
                csv_file = pd.read_csv(temp_file_path, sep=';')    
                shutil.rmtree(temp_dir)
                
            csv_file.iloc[:,0] = csv_file.iloc[:,0].map(lambda x: x[:-3])
            sample_id[i] = list(csv_file.iloc[:,0])
            parameter[i] = list(csv_file.iloc[:,1])
            measured_value[i] = list(csv_file.iloc[:,2])
            
        # sample_id, parameter, datum und measured_value auf dieselbe Dimension bringen
        multi_list = []
        for i in range(len(sample_id)):
            multi_list.append(len(sample_id[i]))

        datum_final = []
        for x,y in zip(datum, multi_list):    
            datum_final.extend([x]*y)

        # flatten the other lists
        sample_final = [entry for sublist in sample_id for entry in sublist]
        parameter_final = [entry for sublist in parameter for entry in sublist]
        meas_value_final = [entry for sublist in measured_value for entry in sublist]            
                
                        
        progress_bar.step(40)
        window.update()

        excel_info = open_dm_file_sheet('Messsysteme und Preise aller Auftragslabore',
                                        'O:/Datenmanagement/Befunde und Messsysteme/Messsysteme/1_Auftragslabore',
                                        'Labor Dr. Spranger')
        excel_info.columns = excel_info.iloc[1].tolist()
        excel_info = excel_info[['Parameter','Parameter Code', 'Anfor-derungs-kürzel', 'Angebots- Preis \n(excl. MwSt)', 'Einheit', 'Referenzbereich', 'Einheit Referenzbereich', 'Bemerkung referenzbereich', 'Methode', 'Gerät', 'Hersteller/ Gerät', 'KIT', 'Hersteller/ KIT']]
        excel_info = excel_info.iloc[2:]
        excel_info = excel_info.iloc[::-1].reset_index().drop('index', axis=1)
        excel_info = excel_info.drop_duplicates(subset='Parameter', keep='last')
        
        progress_bar.step(10)
        window.update()

        # Über Parameterliste von Messsystem-Tabelle iterieren und nach zugehöriger Methode
        # und richtiger Parameterbezeichnung suchen
        abk_liste = list(excel_info['Anfor-derungs-kürzel'])
        abk_liste = [str(abk) for abk in abk_liste]
        parameter_liste = list(excel_info['Parameter'])
        methoden_liste = list(excel_info['Methode'])
        methode = ['']*len(parameter_final)
        for i in range(len(parameter_final)):
            for j in range(len(parameter_liste) - 1, -1, -1):
                if parameter_final[i] in abk_liste[j]:
                    methode[i] = methoden_liste[j]
                    parameter_final[i] = parameter_liste[j]
                    break
        
        # method code identifizieren
        methods_codes_excel = open_dm_file('Methoden Feldcodes', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen')
        codes_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[0]))
        methods_list = list(methods_codes_excel['Methoden'].map(lambda x: x.split('=')[-1]))
        
        progress_bar.step(10)
        window.update()
        
        method_code = ['']*len(methode)
        for i in range(len(methode)):
            for j in range(len(methods_list)):
                if methode[i] in methods_list[j]:
                    method_code[i] = codes_list[j]
                    break
                    
        # measured_value formatieren
        zeichen = ['<', '<=', '>', '>=']
        for i in range(len(measured_value)):
            val = measured_value[i]
            try:
                val = decimal_value(val)
            except:
                for z in zeichen:
                    if z in val:
                        val = val.split(z)[-1]
                        val = decimal_value(val)
                        val = z + ' ' + val
            measured_value[i] = val
            
            
        progress_bar.step(10)
        window.update()

        df = pd.DataFrame({'Sample ID': sample_final,
                           'Datum': datum_final,
                           'Parameter': parameter_final,
                           'Measured value': meas_value_final,
                           'Method code': method_code
                          })
        # join df and excel_info
        df = pd.merge(df, excel_info, on='Parameter')    
        # rearrange columns
        df['Remarks value'] = ['']*len(df)
        df['results taken from'] = ['01']*len(df)
        df['gemessen von'] = ['Labor Dr. Spranger']*len(df)
        new_column_order = ['Sample ID', 'Parameter','Parameter Code', 'Measured value', 'Remarks value',
                            'Einheit', 'Referenzbereich', 'Einheit Referenzbereich',
                            'Bemerkung referenzbereich', 'Methode', 'Method code', 'Gerät',
                            'Hersteller/ Gerät', 'KIT', 'Hersteller/ KIT', 'results taken from',
                            'gemessen von', 'Datum', 'Angebots- Preis \n(excl. MwSt)']
        df = df[new_column_order]
        df = df.sort_values(by=['Datum', 'Sample ID'])
        # Preis 'nach Rücksprache' ersetzen
        df.loc[df['Angebots- Preis \n(excl. MwSt)'] == 'nach Rücksprache', 'Angebots- Preis \n(excl. MwSt)'] = ''

        progress_bar.step(10)
        window.update()
        
        # parameter_final IDA-konform
        parameter_codes_excel = open_dm_file_sheet('Aktueller Laborkatalog', 'O:\Datenmanagement\IDA_in.vent Datenbank\_Vorlagen',
                                              'BioARCHIVE')
        codes = list(parameter_codes_excel['Code'])
        parameter = list(parameter_codes_excel['Title'])
        for i in range(len(df)):
            if df['Parameter Code'][i] in codes:
                code_index = codes.index(df['Parameter Code'][i])
                df['Parameter'][i] = parameter[code_index]
        
        # in Excel-Tabelle einfügen
        add_to_befunddokumentation(df)    

        progress_bar.step(10)
        window.update()
        
        # Ende
        text_info.pack_forget()
        weiter.pack_forget()
        fertig = Label(root2, text = "\n           Fertig! #TeamDodo <3            \n\n",
                       bg = '#eeeee4', font=('Ink free',11))
        fertig.pack()

        # Zurück zur Startseite
        back_to_homepage.grid_forget() # Button von dem Prozessschritt davor
        back_to_homepage = Button(window, text = 'Zurück', bg = '#869287', font=('Ink free', 9, 'bold'),
                        command = go_homepage)
        back_to_homepage.grid(row=20, column=2)
    
    progress_bar.stop()
    progress_bar.grid_forget()

def go_spranger():
    global root2, back_to_homepage, text_info, weiter
    root2 = Frame(window, width=400, height=400, bg = '#eeeee4')
    root2.grid(row=2, column=1, rowspan=5, columnspan=2, pady=2)

    text_spranger = Label(root2, text = "Spranger",
                   bg = '#eeeee4', font=('Ink free',13,'bold'))
    text_spranger.pack()
    text_info = Label(root2, text = "\nBitte klicke auf GO, wenn Befunde vorhanden sind.",
                   bg = '#eeeee4', font=('Ink free',11))
    text_info.pack()
    weiter = Button(root2, text = '  GO  ', bg = '#869287', font=('Ink free', 11, 'bold'),
                   command=spranger_read)
    weiter.pack(pady=(10, 0))
    
    # Unnötige Buttons vom Interface lösen
    labor_28.grid_forget()
    synlab.grid_forget()
    ladr.grid_forget()
    spranger.grid_forget()
    
    # Zurück-Button
    back_to_homepage = Button(window, text = 'Zurück', bg = '#869287', font=('Ink free', 9, 'bold'),
                    command = go_homepage)
    back_to_homepage.grid(row=20, column=2)
    

# Weiter mit User Interface  
# Auswahl Labor
text_labor = Label(window, text="Wähle das entsprechende Labor aus,\n um die pdf-Datei hochzuladen.", font=('Ink free', 12), 
                   bg = '#eeeee4')
text_labor.grid(row=2, column=1, columnspan=2)

labor_28 = Button(window, text = 'Labor 28', width=10, bg ='#869287',
                  font=('Ink free', 11, 'bold'), command = labor28)
labor_28.grid(row=5, column=1, pady=1, columnspan=2)

ladr = Button(window, text = 'LADR', width=10, bg ='#869287',
              font=('Ink free', 11, 'bold'), command = ladr)
ladr.grid(row=6, column=1, pady=1, columnspan=2)

synlab = Button(window, text = 'Synlab', width=10, bg ='#869287', 
                font=('Ink free', 11, 'bold'), command = synlab)
synlab.grid(row=7, column=1, pady=1, columnspan=2)

spranger = Button(window, text = 'Spranger', width=10, bg ='#869287', 
                font=('Ink free', 11, 'bold'), command = go_spranger)
spranger.grid(row=8, column=1, pady=1, columnspan=2)


# Logo
from PIL import ImageTk, Image
frame = Frame(window, width=10, height=10)
frame.grid(row=1, column=0)
img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((100,100)), 
                         master = window)
label = Label(frame, image = img, bg='#eeeee4')
label.pack()

window.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\l.nguyen quoc\Anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\l.nguyen quoc\AppData\Local\Temp\ipykernel_105616\1844927314.py", line 2150, in spranger_read
    add_to_befunddokumentation(df)
  File "C:\Users\l.nguyen quoc\AppData\Local\Temp\ipykernel_105616\1844927314.py", line 116, in add_to_befunddokumentation
    ws.cells(start_row + r_idx, c_idx).value = cell_value
KeyboardInterrupt
